In [1]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""Convolutional Neural Network Estimator for MNIST, built with tf.layers."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  # Reshape X to 4-D tensor: [batch_size, width, height, channels]
  # MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  # Computes 32 features using a 5x5 filter with ReLU activation.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 28, 28, 1]
  # Output Tensor Shape: [batch_size, 28, 28, 32]
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  # First max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 28, 28, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 32]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2
  # Computes 64 features using a 5x5 filter.
  # Padding is added to preserve width and height.
  # Input Tensor Shape: [batch_size, 14, 14, 32]
  # Output Tensor Shape: [batch_size, 14, 14, 64]
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #2
  # Second max pooling layer with a 2x2 filter and stride of 2
  # Input Tensor Shape: [batch_size, 14, 14, 64]
  # Output Tensor Shape: [batch_size, 7, 7, 64]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Flatten tensor into a batch of vectors
  # Input Tensor Shape: [batch_size, 7, 7, 64]
  # Output Tensor Shape: [batch_size, 7 * 7 * 64]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

  # Dense Layer
  # Densely connected layer with 1024 neurons
  # Input Tensor Shape: [batch_size, 7 * 7 * 64]
  # Output Tensor Shape: [batch_size, 1024]
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

  # Add dropout operation; 0.6 probability that element will be kept
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits layer
  # Input Tensor Shape: [batch_size, 1024]
  # Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main(unused_argv):
  # Load training and eval data
  mnist = tf.contrib.learn.datasets.load_dataset("mnist")
  train_data = mnist.train.images  # Returns np.array
  train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
  eval_data = mnist.test.images  # Returns np.array
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

  # Create the Estimator
  mnist_classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

  # Set up logging for predictions
  # Log the values in the "Softmax" tensor with label "probabilities"
  tensors_to_log = {"probabilities": "softmax_tensor"}
  logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

  # Train the model
  train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=100,
      num_epochs=1,
      shuffle=True)
  mnist_classifier.train(
      input_fn=train_input_fn,
      steps=500,
      hooks=[logging_hook])

  # Evaluate the model and print results
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": eval_data}, y=eval_labels, num_epochs=1, shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)


if __name__ == "__main__":
  tf.app.run()

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_ch

INFO:tensorflow:loss = 1.5510373, step = 1268
INFO:tensorflow:probabilities = [[0.03172066 0.06547987 0.07271157 0.03344529 0.25449127 0.10034301
  0.07122888 0.14841641 0.09387287 0.12829013]
 [0.4721431  0.01416328 0.05599545 0.08274173 0.03605578 0.11422236
  0.03500862 0.07045864 0.06454486 0.05466608]
 [0.05520637 0.03557976 0.07993352 0.07712342 0.09345885 0.04799385
  0.42509294 0.05277321 0.05149917 0.08133887]
 [0.13168426 0.10950211 0.24955037 0.15056242 0.02227276 0.11111304
  0.02876887 0.03135521 0.12609516 0.03909574]
 [0.12078317 0.04257131 0.03746584 0.0263131  0.26086515 0.05360794
  0.13089752 0.09209079 0.10916099 0.12624425]
 [0.09169958 0.04507152 0.15687545 0.0904705  0.09963096 0.05988446
  0.3034929  0.01959267 0.0936527  0.03962933]
 [0.3765275  0.01513583 0.08700451 0.07344706 0.0542319  0.13415876
  0.08014072 0.08647805 0.05819076 0.03468485]
 [0.03636296 0.11584357 0.11711662 0.14106117 0.08323465 0.09233896
  0.09324475 0.11050407 0.10834833 0.10194495]
 [

INFO:tensorflow:global_step/sec: 0.403942
INFO:tensorflow:probabilities = [[0.04335545 0.0498975  0.10141765 0.04962381 0.09136477 0.05695798
  0.43999755 0.02713753 0.10089956 0.03934821]
 [0.06349942 0.02410082 0.0145087  0.07071419 0.09129169 0.07344618
  0.03176086 0.43340933 0.04107198 0.15619683]
 [0.05985857 0.01470628 0.02822592 0.04033905 0.47824022 0.05834315
  0.09694941 0.0615342  0.04748602 0.11431724]
 [0.0663937  0.04330848 0.02747428 0.05660968 0.04348332 0.05900577
  0.02139485 0.3981438  0.08420425 0.19998185]
 [0.07543452 0.0659107  0.07226744 0.09759365 0.14292133 0.16526414
  0.12158768 0.05220684 0.13421537 0.07259836]
 [0.10009361 0.06616797 0.07914942 0.04252147 0.14422688 0.12958083
  0.05232552 0.09085514 0.16106062 0.13401859]
 [0.02035822 0.3327064  0.09083127 0.0941073  0.04050201 0.07270609
  0.09921503 0.07953864 0.11137214 0.05866288]
 [0.05408561 0.0164479  0.5564235  0.14442855 0.01445097 0.06516294
  0.03254223 0.01236346 0.09664943 0.00744534]
 [0.03

INFO:tensorflow:loss = 1.2504982, step = 1368 (247.563 sec)
INFO:tensorflow:probabilities = [[0.02732046 0.28372297 0.1539196  0.14065197 0.02303427 0.04465875
  0.04455886 0.12071934 0.11336062 0.0480531 ]
 [0.11281342 0.01590399 0.1003482  0.51512235 0.0045007  0.03728082
  0.09933025 0.01479806 0.02724361 0.0726586 ]
 [0.5674681  0.00300673 0.02867537 0.08653796 0.03332435 0.04449921
  0.07689074 0.07593005 0.05567281 0.02799474]
 [0.03091514 0.0692231  0.03871815 0.05751646 0.066317   0.04012588
  0.0343496  0.44179982 0.066902   0.15413275]
 [0.12886047 0.01171775 0.03899711 0.07733654 0.15152855 0.05230244
  0.03274938 0.13806942 0.08969312 0.27874523]
 [0.05958926 0.02747391 0.03216058 0.11384831 0.0807156  0.04063933
  0.02064491 0.36398596 0.05308191 0.20786023]
 [0.04543437 0.01047159 0.1290281  0.03123724 0.08891376 0.03189896
  0.56653446 0.0243449  0.03856004 0.03357661]
 [0.05478965 0.10754901 0.25767693 0.04756117 0.04023609 0.12732455
  0.09129529 0.0374882  0.19271947 

INFO:tensorflow:global_step/sec: 0.389379
INFO:tensorflow:probabilities = [[0.11320786 0.06371708 0.12171349 0.22524637 0.00631703 0.10606268
  0.01619965 0.17490347 0.14645025 0.02618212]
 [0.03172492 0.05264944 0.1118303  0.3926437  0.03654838 0.05687824
  0.03795027 0.09309207 0.11243352 0.07424925]
 [0.00721924 0.00251266 0.01839596 0.01878743 0.6116553  0.02660174
  0.02765952 0.02917472 0.06086252 0.19713092]
 [0.920142   0.00086774 0.02189278 0.00670714 0.00857725 0.01483978
  0.0071309  0.01258889 0.00469854 0.00255488]
 [0.01734271 0.24468206 0.05627304 0.05939752 0.03034223 0.05784449
  0.06142359 0.19418396 0.21610731 0.06240316]
 [0.01998048 0.4114646  0.06781131 0.06326646 0.04701914 0.05034914
  0.06037468 0.07241273 0.11338637 0.09393506]
 [0.05148894 0.00813383 0.20870955 0.42509544 0.10083685 0.08728081
  0.04619956 0.00589063 0.03672372 0.02964071]
 [0.0602464  0.06837223 0.13237573 0.4312895  0.01871583 0.11205019
  0.01379995 0.05587593 0.07106825 0.03620601]
 [0.04

...

INFO:tensorflow:global_step/sec: 0.379127
INFO:tensorflow:probabilities = [[0.03605955 0.01508573 0.00257936 0.02168494 0.01307368 0.0911235
  0.00652101 0.6059464  0.10651003 0.10141577]
 [0.06370908 0.00034183 0.02588235 0.00266056 0.5520405  0.04496223
  0.21549147 0.02538636 0.00387882 0.06564683]
 [0.00326196 0.50032383 0.02186794 0.08395134 0.01253861 0.02081042
  0.03200104 0.06593101 0.07909183 0.18022205]
 [0.00389226 0.06982026 0.03438883 0.5310146  0.00776967 0.03597477
  0.0118018  0.10147011 0.09407382 0.10979386]
 [0.93487376 0.00006854 0.00456224 0.01737307 0.00115947 0.02606913
  0.00067077 0.00113343 0.01283559 0.00125393]
 [0.9750113  0.00010855 0.0050421  0.00264852 0.00131379 0.00945757
  0.00310484 0.00087254 0.00177313 0.0006678 ]
 [0.01823408 0.00735318 0.14118685 0.49354663 0.01395705 0.03463316
  0.01322474 0.04458359 0.2098271  0.02345361]
 [0.18749838 0.00281895 0.01259685 0.01585707 0.09688887 0.17567964
  0.00669091 0.13720852 0.23366058 0.13110018]
 [0.095

INFO:tensorflow:loss = 0.94254357, step = 1668 (263.835 sec)
INFO:tensorflow:probabilities = [[0.01716277 0.06249686 0.01536917 0.02674691 0.04954966 0.06516398
  0.0214638  0.11099324 0.39116657 0.23988706]
 [0.01438989 0.04187404 0.0360927  0.05371836 0.21372323 0.15379114
  0.07854719 0.13199408 0.05891283 0.21695654]
 [0.03463648 0.00250328 0.77148116 0.04773614 0.0057273  0.06894393
  0.04705783 0.00099542 0.01617482 0.00474352]
 [0.01004118 0.04554861 0.01695666 0.00969956 0.12944092 0.03570239
  0.01392318 0.11522219 0.14694282 0.47652248]
 [0.03583926 0.01113126 0.02582351 0.033208   0.22676015 0.10217488
  0.02711888 0.3387948  0.01916252 0.17998676]
 [0.05472492 0.008836   0.0888503  0.02453196 0.48811695 0.04389064
  0.08849304 0.01882152 0.02157186 0.16216289]
 [0.87907165 0.00013323 0.00416257 0.00446868 0.00401512 0.0887251
  0.00482679 0.00053127 0.01098624 0.00307926]
 [0.05151931 0.04983141 0.140431   0.3498814  0.02453974 0.04294463
  0.05831651 0.01602249 0.17163804 

INFO:tensorflow:Saving checkpoints for 1731 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:Saving checkpoints for 1767 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:Loss for final step: 0.94982314.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-02-24-09:49:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model\model.ckpt-1767
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-02-24-09:49:38
INFO:tensorflow:Saving dict for global step 1767: accuracy = 0.8414, global_step = 1767, loss = 0.7702712
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1767: /tmp/mnist_convnet_model\model.ckpt-1767
{'accuracy': 0.8414, 'loss': 0.7702712, 'global_step': 1767}


SystemExit: 

c:\users\jayanth\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
%tb

SystemExit: 